In [6]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from functions import mean_gen, varx_gen, vary_gen, spherical_data, test_on_integers, learn_parameters

In [7]:
# Random Seed
rand_n = 12345

# Single Point on Sphere
n = 40000
angles = np.array([2.5, 2.5])
thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

In [8]:
model_interpolate = tf.keras.models.load_model('3dmodels/discrete_model_mth40_mph40')

for l in model_interpolate.layers:
    l.trainable=False
    
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold = tf.keras.Input(shape=(1,))
simple_linear = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs = tf.keras.Input(shape=(2,))
inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
output = model_interpolate(inputs)

model_paramfinder = Model(inputs = raw_inputs, outputs = output)
model_paramfinder.compile(loss=loss_fn, optimizer='Adam')
model_paramfinder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like_1 (TFOpLambda)     (None, 2)            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 1)            0           tf.ones_like_1[0][0]             
__________________________________________________________________________________________________
model_2 (Functional)            (None, 2)            2           tf.__operators__.getitem_1[0][0] 
____________________________________________________________________________________________

In [9]:
test_on_integers(x_train, y_train)

discrete_model_th5_ph3 has loss 0.6399607412144142
discrete_model_th3_ph3 has loss 0.6243008177038691
discrete_model_th2_ph2 has loss 0.6364817954972899
discrete_model_th1_ph1 has loss 0.7387577053283457
discrete_model_th0_ph0 has loss 0.6932041742764413


In [ ]:
epochs, iterations, batch_size = 1, 50, 100

weights = learn_parameters(model_paramfinder, x_train, y_train, iterations, epochs, batch_size)
angle_errors = (angles - weights) / angles
average_error = np.sum(np.abs(angle_errors)) / 2
print('Average error %:', average_error * 100)

640/640 [==============================] - 2s 2ms/step - loss: 0.6811
0 Fitted result:  tf.Tensor([1.4862916 1.1815978], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.6453
1 Fitted result:  tf.Tensor([1.8383607 1.3018242], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.6304
2 Fitted result:  tf.Tensor([2.106011  1.4494157], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.6216
3 Fitted result:  tf.Tensor([2.288649  1.7635429], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.6124
4 Fitted result:  tf.Tensor([2.3843012 2.139596 ], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.6082
5 Fitted result:  tf.Tensor([2.402034 2.333947], shape=(2,), dtype=float32)
640/640 [==============================] - 1s 2ms/step - loss: 0.6076
6 Fitted result:  tf.Tensor([2.4033055 2.425

In [5]:
xr = model_interpolate(x_train).numpy().transpose()[0]
y = model_interpolate(x_train)
loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
print(loss)

0.6319640082872647


In [ ]:
sort(xr)

In [11]:
x_train_mod = np.copy(x_train)
x_train_mod[:, 2] = x_train[:, 2] - .1
x_train_mod[:, 3] = x_train[:, 3]
xr_mod = model_interpolate(x_train_mod).numpy().transpose()[0]
y_mod = model_interpolate(x_train_mod)
loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
print(loss_mod)

0.6298558847219974


In [25]:
x_train

array([[ 1.07633646e-01,  1.70760504e+00,  4.00000000e+00,
         3.00000000e+00],
       [ 1.43298843e+00,  3.84883422e-01,  4.00000000e+00,
         3.00000000e+00],
       [ 3.76967497e-02,  7.15595959e-01,  4.00000000e+00,
         3.00000000e+00],
       ...,
       [ 2.12611480e-03,  6.37581970e-01,  4.00000000e+00,
         3.00000000e+00],
       [ 8.48878054e-01,  5.50897652e-01,  4.00000000e+00,
         3.00000000e+00],
       [-9.53562972e-01,  5.37223246e-02,  4.00000000e+00,
         3.00000000e+00]])